In [1]:
import subprocess
import timeit
import os
import time
import math
import random
from memory_profiler import memory_usage
from guppy import hpy; h=hpy()
import pygad
import numpy


In [2]:
class hill_climb:
    
    class climber:
        
        def __init__(self):
            
            self.combo = self.make_rand_position()
            self.combo = [0]*150
            self.fitness = 0
            self.n_evals = 0
        def show(self):
            
            print(self.combo)
        
        def evaluate(self, combo):

            f=open('input.txt','w+')
            for item in combo:
                f.write(str(item)+'\n')
                
            f.close()

            os.system('./cpp_eval')
            
            f = open('fitness_out.txt', 'r')
            fitness = f.read()
            f.close()
            
            self.n_evals = self.n_evals + 1
            
            #print('fit',fitness)
            return fitness
        
        def update(self,combo):
            fitness = self.evaluate()
            #print('fit',fitness)

            return fitness

        def make_rand_position(self):
            rand_position = [0]*150
            for i in range(0,150):
                rand_position[i] = str(int(random.random() > 0.5))
            return rand_position

        def inject_noise(self,position):
            new_position = position.copy()
            noise = self.make_rand_position()
            for i in range(0,150):
                if int(noise[i]):
                    new_position[i] = str(int(not int(new_position[i])))
            #print(noise)
            #print(position)
            return new_position
        
        def map_differentials(self,position):
            diff_list = []

            for i in range(0,150):

                row = position.copy()
                row[i] = int(not int(row[i]))
                #print('position', i, 'from' , position[i],'to',row[i] )
                diff_list.append(row)

            return diff_list

        def update_differential(self,diff_list, previous_position):

            i = 0
            starting_fitness = self.evaluate(previous_position)

            differential_all_dimensions = {}

            for row in diff_list:
                fitness = self.evaluate(row)
                differential =  int(fitness) -int(starting_fitness) 
                differential_all_dimensions.update({str(i): differential})
                i = i +1

            return differential_all_dimensions


        def iterative_solver( self, init_position, n_iterations):
            self.climber_stuck = False
            self.known_keys = []
            previous_position = init_position.copy()
            best_known_fitness = self.evaluate(init_position)
            
            for iterations in range(n_iterations):
                
                if self.climber_stuck is False:

                    new_position = previous_position.copy()

                    for key in range(150):
    
                        candidate_position = new_position.copy()
                        candidate_position[key] = int(not new_position[key])
                        candidate_fitness = self.evaluate(candidate_position)
                
                        if candidate_fitness > best_known_fitness:
                            self.known_keys.append(key)
                            
                            if candidate_fitness == 150:
                                return candidate_fitness
                            
                            new_position = candidate_position
                            best_known_fitness = candidate_fitness
                            print('fitness',best_known_fitness)
                            
                    if previous_position == new_position:
                        self.climber_stuck = True
                        
                    previous_position = new_position
                    
                else:
                    print('stuck at:', previous_position)
                    #enumerate rest of state space to search

                    self.unknown_keys = []

                    self.unknown_keys = list(set(range(150)) -set(self.known_keys) )
                    
                    #check all bits not checked
                    for key in self.unknown_keys:
                        candidate_position = previous_position.copy()
                        candidate_position[key] = int(not previous_position[key])
                        candidate_fitness = self.evaluate(candidate_position)
                        if candidate_fitness < best_known_fitness:
                            self.known_keys.append(key)
                            
                    self.unknown_keys = list(set(range(150)) -set(self.known_keys) )
                    
                    n_unknowns = len(self.unknown_keys)
                    
                    search_vector_max = 2**len(self.unknown_keys)
    
                    search_vector = 0
                    solved = False
                    
                    candidate_position = previous_position.copy()
                    print('unkown terms', len(self.unknown_keys),'max search terms',search_vector_max)
                    
                    sub_iterations = 15
                    
                    for s_i in range(sub_iterations):
                        candidate_bits = format(search_vector, 'b')
                        #pad_zeros 
                        candidate_bits = candidate_bits.zfill(n_unknowns)
                        print(candidate_bits)
                        
                        for i in range(n_unknowns):
                            candidate_position[i] = candidate_bits[i]
                        
                        candidate_fitness = self.evaluate(candidate_position)
                        
                        if candidate_fitness > best_known_fitness:
                            previous_position = candidate_position
                            best_known_fitness = candidate_fitness
                        
                        search_vector = math.floor(search_vector +  search_vector_max *\
                        ( int(candidate_fitness )-int(best_known_fitness) ) \
                        / int(best_known_fitness)**2)
                        
                        if search_vector == search_vector_max:
                            solved = True
                            
                        if candidate_fitness == 150:
                            return candidate_fitness
                        #candidate_position = previous_position.copy()
                        
                        
                    
           
            #print('kk',(self.known_keys),'\nuk', (self.unknown_keys))
                    
            return [best_known_fitness, previous_position]
                    

In [3]:
solution = []

In [4]:
my_climber = hill_climb.climber()
    
n_trials = 10

for i in range(n_trials):
    
    #init_pos = my_climber.make_rand_position()
    #solution.append(my_climber.iterative_solver(init_pos, 4))
    print('foo')
    print(h.heap())

foo
Partition of a set of 440297 objects. Total size = 53612934 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 121671  28 15413577  29  15413577  29 str
     1 124249  28  9475264  18  24888841  46 tuple
     2  52775  12  4033716   8  28922557  54 bytes
     3  26517   6  3833944   7  32756501  61 types.CodeType
     4  25407   6  3455352   6  36211853  68 function
     5   3208   1  3148736   6  39360589  73 type
     6   7375   2  2772648   5  42133237  79 dict (no owner)
     7   1056   0  1620800   3  43754037  82 dict of module
     8   3208   1  1595992   3  45350029  85 dict of type
     9   2532   1   881024   2  46231053  86 set
<1073 more rows. Type e.g. '_.more' to view.>
foo
Partition of a set of 440306 objects. Total size = 53613682 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 121671  28 15413577  29  15413577  29 str
     1 124250  28  9475320  18  24888897  46 tuple
     2  52775  12  40

In [5]:
print('Evaluations per solution = ' ,my_climber.n_evals/n_trials)

Evaluations per solution =  0.0


In [6]:
print

<function print>

In [ ]:
"""
Given the following function:
    y = f(w1:w6) = w1x1 + w2x2 + w3x3 + w4x4 + w5x5 + 6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7) and y=44
What are the best values for the 6 weights (w1 to w6)? We are going to use the genetic algorithm to optimize this function.
"""

function_inputs = [0]*150 # Function inputs.
desired_output = 44 # Function output.

def fitness_func(solution, solution_idx):
    # Calculating the fitness value of each solution in the current population.
    # The fitness function calulates the sum of products between each input and its corresponding weight.
    print(solution)
    output = numpy.sum(solution*function_inputs)
    fitness = int(my_climber.evaluate(solution))
    #fitness = 1.0 / numpy.abs(output - desired_output)
    print(fitness)
    return fitness

fitness_function = fitness_func

num_generations = 100 # Number of generations.
num_parents_mating = 7 # Number of solutions to be selected as parents in the mating pool.

# To prepare the initial population, there are 2 ways:
# 1) Prepare it yourself and pass it to the initial_population parameter. This way is useful when the user wants to start the genetic algorithm with a custom initial population.
# 2) Assign valid integer values to the sol_per_pop and num_genes parameters. If the initial_population parameter exists, then the sol_per_pop and num_genes parameters are useless.
sol_per_pop = 50 # Number of solutions in the population.
num_genes = 150

init_range_low = 0
init_range_high = 2

parent_selection_type = "sss" # Type of parent selection.
keep_parents = 7 # Number of parents to keep in the next population. -1 means keep all parents and 0 means keep nothing.

crossover_type = "single_point" # Type of the crossover operator.

# Parameters of the mutation operation.
mutation_type = "random" # Type of the mutation operator.
mutation_percent_genes = 10 # Percentage of genes to mutate. This parameter has no action if the parameter mutation_num_genes exists or when mutation_type is None.

last_fitness = 0
def callback_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution()[1] - last_fitness))
    last_fitness = ga_instance.best_solution()[1]

# Creating an instance of the GA class inside the ga module. Some parameters are initialized within the constructor.
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating, 
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop, 
                       num_genes=num_genes,
                       init_range_low=init_range_low,
                       init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       callback_generation=callback_generation,
                       gene_type = int,
                       gene_space = [0,1])

# Running the GA to optimize the parameters of the function.
ga_instance.run()
print(h.heap())
# After the generations complete, some plots are showed that summarize the how the outputs/fitenss values evolve over generations.
ga_instance.plot_result()

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution()
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

prediction = numpy.sum(numpy.array(function_inputs)*solution)
print("Predicted output based on the best solution : {prediction}".format(prediction=prediction))

if ga_instance.best_solution_generation != -1:
    print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))

# Saving the GA instance.
filename = 'genetic' # The filename to which the instance is saved. The name is without extension.
ga_instance.save(filename=filename)

# Loading the saved GA instance.
loaded_ga_instance = pygad.load(filename=filename)
loaded_ga_instance.plot_result()


Starting from PyGAD 2.6.0, the callback_generation parameter is deprecated and will be removed in a later release of PyGAD. Please use the on_generation parameter instead.
[0 0 1 1 0 0 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 0 0 1 0 0
 0 0 1 0 1 0 0 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 1 0 0 0 0 0
 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 0
 1 0 1 1 0 1 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 1 1 1 0 0 0
 0 0]
32
[0 1 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 1 1 1 0 1 0 1 1 0
 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0 1 0 1 0 0 1 1 0
 0 0 0 1 0 0 0 1 1 0 1 0 0 1 1 1 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 1 1 0 1 0 1
 1 1 1 1 1 1 1 0 0 0 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 1 1 1
 0 0]
30
[1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 0 0 1 0 1 0 0 0 0 0 1 1 0 1 1 1 1
 1 1 0 0 0 1 0 1 1 0 0 1 1 1 0 1 1 1 0 0 0 0 1 0 1 0 1 1 1 0 0 1 0 1 1 0 0
 1 0 0 0 0 1 1 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0

32
[0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 1 1 1 0 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 0 0 1
 1 1 0 1 1 1 0 1 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0 0 1
 1 0 1 0 0 1 0 0 1 0 1 0 0 1 1 0 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0
 1 0 0 0 0 0 1 1 0 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 1 1 1 0 1 1 1
 1 1]
32
[0 0 0 1 1 0 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1
 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 0 0 1 1 0
 1 1 0 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 1 0 0 0
 1 1 1 1 1 0 0 1 0 1 0 0 1 0 1 0 0 1 1 1 1 1 0 1 0 1 0 1 0 0 1 0 0 1 0 0 0
 0 1]
33
[1 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 0 1 1 1 0
 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 1 1 1
 1 1 0 1 0 0 0 1 1 0 1 0 1 1 0 0 1 0 1 1 0 0 0 1 1 1 0 0 1 1 0 0 1 1 1 0 1
 0 1 0 1 1 0 0 1 1 1 0 1 0 1 0 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0
 0 0]
32
[1 1 0 1 0 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 0 0 1 1 1 0 1 0

32
[1 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 1 0 0 1 1 0 0 1 1 0
 1 1 0 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0 1 0 1 1
 1 0 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 1 1 0 0 0 0 0 0 1 0
 0 1]
32
[1 0 0 1 1 1 0 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 1
 0 1 0 0 1 1 0 0 1 0 1 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 1 1 1 0 0 1 0 0 0 0
 1 0 1 0 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0
 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0
 0 1]
32
[1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1
 0 0 1 1 1 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 1
 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 0 0 0 0 1 1 0 0 1 1 1
 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 0 0 1 1 1 0 0 0 1
 0 0]
32
[1 1 0 0 0 0 0 1 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 1

32
[0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0
 0 1 0 1 1 1 1 0 1 1 0 1 1 1 1 0 1 0 1 0 1 0 1 1 1 0 0 1 0 0 1 0 0 0 1 1 1
 0 1 0 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1
 1 0 0 1 0 1 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 1 1 0 0 0 0 0 0 0 0
 0 1]
32
[1 1 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1 1 0
 1 1 0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 1 1 0 1 1 1 0 0 1 0 0 0 0 0 1 1 0 0 1 0
 1 0 1 1 1 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0
 1 0 1 1 1 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 1 0 0 0 0 1 0 0 1 0 0
 0 1]
32
[1 0 0 1 1 1 0 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 1 1
 0 0 1 1 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 1 1 1 1 1 1 0 0 1 1 0 0 1 0 1 1
 0 1 1 1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 0 0 0 1 1 0 1 1 1 1
 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0 0 0 0
 0 0]
32
[1 1 1 0 0 1 1 0 0 1 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 0 0 1 0 1 1 1 1 0

32
[0 0 0 1 1 0 1 0 0 1 0 1 1 1 1 0 1 1 0 0 1 0 0 0 0 0 0 1 1 1 1 0 1 0 0 0 1
 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0
 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 1 0 1
 1 0 0 0 0 1 0 1 0 1 1 1 1 1 0 1 1 0 1 0 0 0 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1
 1 1]
32
[0 0 0 1 0 0 1 1 0 1 0 0 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1
 0 1 1 1 1 0 1 0 1 1 0 1 1 1 0 0 1 0 1 1 1 0 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1
 0 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 1 1 1 1 1 0 1
 0 1 0 0 1 1 1 1 0 0 1 0 1 0 0 0 1 0 1 1 1 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 0
 0 1]
32
[0 0 1 0 0 1 1 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 1 0 1 0 0 1 1 1 0 1 1 0
 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 0 1 0 1 0 1 0 1 1 1 0 0 1 0 0 1 0 0 0 1 0 1
 0 1 0 1 1 1 1 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 1 1
 1 0 0 1 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1 1 1 1 1 0 0 0 0 0 0 1 0
 0 1]
32
[1 0 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1

...

32
[0 0 1 0 0 1 1 0 1 1 0 0 1 1 1 0 1 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1
 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 0 0 1
 1 1 0 0 1 1 1 1 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0
 0 0 0 0 1 1 1 1 0 0 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 1 0 0
 0 1]
44
[0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 1 1 1 0 0 1 1 0 1 1
 0 1 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1
 1 1 0 0 1 1 0 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0
 0 0 0 1 1 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 1 0 1 0 0 0
 0 1]
40
[0 0 0 1 0 0 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 1 0 0 1 0 0 1 1 0 0 1 1 0 1 1
 1 1 0 0 0 1 0 1 0 0 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1
 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 0 0 1 1 0 1 1 0 0
 0 0 0 1 1 1 1 1 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0 0 1 0 0 0
 1 1]
32
[0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 1 1 0 1

64
[0 0 0 0 0 1 0 0 1 1 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1
 1 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 1 1 1
 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0
 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 1 0 0
 1 1]
32
[0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1
 1 1 0 0 0 1 0 1 1 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 1 1 1
 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0
 0 0 0 1 1 0 1 1 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0 1 1 0 1 0 0 0
 0 1]
64
[0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1
 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 0 0 0 0 1 1 0 1 0 0 1
 1 1 0 0 1 1 1 1 1 0 0 1 0 1 0 1 1 0 1 0 1 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0
 0 0 0 0 1 1 0 1 1 1 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 1 0 0
 1 1]
64
[0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 0 1 1 1

36
[0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1
 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 0 0 0 0 1 0 0 1 1 1
 1 1 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0 1 1 1 0 1 1 1 0 1 0 0 0
 1 0 1 1 0 1 0 1 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0
 0 1]
48
[0 0 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1
 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0 1 0 1
 1 1 0 0 1 0 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0
 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0
 0 1]
32
[0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 1 1 1 1 0 0 1 0 1 1 1 0 0 1 1 1 1 1
 1 1 1 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 1
 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0
 0 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 1 0 0
 1 1]
36
[0 0 1 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 0 1 1 1

64
[0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 0 1 1
 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 0 0 0 0 1 1 0 0 1 0 1
 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1 0 1 1 1 0 0 1 1 0 1 0 0 0
 0 0 0 0 1 1 0 1 1 0 0 0 0 1 1 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 1 0 0
 1 1]
64
[0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 1 1
 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 0 0 0 1 1 1 1 1 0 1 0 0 1 0 0 0 1 0 1
 0 1 0 0 1 1 0 0 0 0 1 1 0 1 1 1 1 0 1 0 0 1 1 0 0 1 0 1 0 0 1 1 0 1 0 0 0
 0 0 0 1 1 1 1 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0 1 1 0 1 0 0 0
 1 1]
64
[0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0 1 0 0 1 1 0 0 1 1 1 1 1
 1 1 0 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 1 1 1
 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 1 0 0 1 1 0 1 0 0 0
 0 0 0 1 1 1 1 1 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0 1 1 0 1 0 0 0
 0 1]
64
[0 0 0 0 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 0 1 0 0 1 1 0 0 1 1 1

44
[0 0 0 1 0 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 1 0 1 1 0 0 1 1 0 0 1 1 1 1 1
 1 1 0 1 0 1 0 1 0 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 1 1
 1 1 0 1 1 1 1 0 1 0 0 0 1 1 1 1 1 0 1 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 0
 1 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 0 1 0 1 1 0 1
 1 1]
32
[0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 1 0 1 0 1 0 1 1
 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 1 1 1 0 0 0 0 1 1 0 0 1 0 0
 1 1 0 0 1 1 0 1 1 0 1 1 0 1 1 1 1 0 0 0 0 1 1 0 0 1 0 1 0 0 1 1 0 1 0 0 0
 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 1 0 1 1 0 0 0 1 1 1 1 0 0 0
 0 1]
32
[0 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 1 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1 1 1
 1 1 0 0 0 0 0 1 1 0 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 1
 1 1 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 0 1 1 0 0 0 1 0 1 1 1 0 0 1 1 0 1 0 0 1
 0 0 0 1 1 1 1 1 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 1 0 0 0 1 1 0 1 0 0 0
 0 1]
32
[0 0 0 0 1 0 0 0 1 1 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 1 1 1

In [ ]:
print('number of evals; ', my_climber.n_evals)